In [1]:
import cv2 as cv
import pathlib
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [10]:
pip list

Package                      Version
---------------------------- -----------
absl-py                      1.4.0
affine                       2.4.0
aiohttp                      3.9.5
aiohttp-cors                 0.7.0
aiosignal                    1.3.1
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.3
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
boto3                        1.34.101
botocore                     1.34.101
bqplot                       0.12.42
branca                       0.7.0
cachetools                   5.3.1
certifi                      2023.7.22
cffi                         1.16.0
charset-normalizer           3.2.0
click                        8.1.7
click-plugins                1.1.1
cligj                        0.7.2
cloudpickle                  2.2.1
colorama                     0.4.6
colorful                     0.5.6
colour                       0.1


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: C:\Users\Owner\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
# cap = cv.VideoCapture(0)
cap = cv.VideoCapture("bricksimulator.mp4")
window = "Window name"
cv.namedWindow(window)



# ORANGE RANGE (FOUND MANUALLY)
orange_upper = np.array([15, 255, 255])
orange_lower = np.array([5, 100, 100])
 
centers = []
counter = [0]

# instantiate simpleblobdetector
params = cv.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 0
params.maxThreshold = 25


# Filter by Area.
params.filterByArea = True
params.minArea = 500
params.maxArea = 1000

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.45
params.maxCircularity = 0.55

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.85
params.maxConvexity = 1

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01
params.maxInertiaRatio = 0.1



sbd = cv.SimpleBlobDetector_create(params)

rim_blobs = []

# frame counter
frame_counter = 0

# holder for rim coordinates and radius
rim = []

# main function that is run every frame
def perf_op(name, frame, centers, counter, rim_blobs):

    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    global frame_counter
    global sbd

    # specifies number of frames used to find the rim
    # since it only needs to be found once
    rim_finding_frame = 5

    # rim finding algorithm
    if len(rim_blobs) < rim_finding_frame:
        # rim color values
        rimlower, rimupper = np.array([rimval1, rimval3, rimval5]), np.array([rimval2, rimval4, rimval6])
        # narrows image down to rim colors
        rim_mask = cv.inRange(hsv, rimlower, rimupper)
        rim_filtered = cv.bitwise_and(frame, frame, mask = rim_mask)
        rim_rgb = cv.cvtColor(rim_filtered, cv.COLOR_HSV2RGB)
        rim_grayscl = cv.cvtColor(rim_rgb, cv.COLOR_RGB2GRAY)

        # blurs to make rim's outline bigger
        blurred = cv.GaussianBlur(255 - rim_grayscl, (15, 15), 0)
        ret, binary = cv.threshold(blurred, 250, 255, cv.THRESH_BINARY)

        # finds the rim as a blob
        rim_blobs.append(sbd.detect(255 - binary)[0])

    # fills rim array after the specified number of frames
    if frame_counter == rim_finding_frame:
        # uses median values in case of weird outliers
        rim_xs = np.array(list(map(lambda x: x.pt[0], rim_blobs)))
        rim_ys = np.array(list(map(lambda x: x.pt[1], rim_blobs)))
        rim_rads = np.array(list(map(lambda x: x.size, rim_blobs)))
        rim.append(int(np.median(rim_xs)))
        rim.append(int(np.median(rim_ys)))
        rim.append(int(np.median(rim_rads)))

    # finds orange circle (aka the basketball)

    lower, upper = np.array([val1, val3, val5]), np.array([val2, val4, val6])
    mask = cv.inRange(hsv, lower, upper)


    filtered = cv.bitwise_and(frame, frame, mask = mask) 

    rgb = cv.cvtColor(filtered, cv.COLOR_HSV2RGB)
    grayscl = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
    circles = cv.HoughCircles(grayscl, cv.HOUGH_GRADIENT, 1.5, 5000, param2 = 20, minRadius = 10)
    if circles is not None:
        counter[0] = 0
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv.circle(frame,(i[0],i[1]),i[2],(15,255,255),2)
            # draw the center of the circle
            if len(centers) == 0:
                centers.append(i)
            elif ((centers[-1][0] - i[0])**2 + (centers[-1][1] - i[1])**2 )**.5 > centers[-1][2] * 4:
                cv.circle(frame, (500, 500), 200, (0, 255, 0), 2)
                continue
            else:
                centers.append(i)
    else:
        counter[0] += 1
        if counter[0] == 5:
            centers.clear()
            counter[0] = 0

    # ignore debug comments
    for i in range(len(centers)):
        if i != 0:
            cv.line(frame, centers[i][:2], centers[i-1][:2], (255, 0, 0), 2)
        cv.circle(frame, centers[i][:2], 2, (5, 92, 94), 3)
        if not (centers[i][0] - rim[0])**2 + (centers[i][1] - rim[1])**2 < rim[2]**2 * 1.44:
            #cv.putText(frame, "SHOT", (rim[0], rim[1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
            # if (centers[i-1][0] - rim[0])**2 + (centers[i-1][1] - rim[1])**2 > rim[2]**2* 1.44:
            #     cv.putText(frame, "IN", (centers[i][0], centers[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
            if i != 0 and (centers[i-1][0] - rim[0])**2 + (centers[i-1][1] - rim[1])**2 < rim[2]**2* 1.44:
                # cv.putText(frame, "OUT", (centers[i][0], centers[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                # cv.putText(frame, str(i-1), (centers[i][0] + 100, centers[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                ang = np.arctan2([centers[i][1] -rim[1]], [centers[i][0] - rim[0]])[0]
                # cv.putText(frame, str(rim), (centers[i][0], centers[i][1] + 100), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                # cv.putText(frame, str(centers[i]), (centers[i][0] + 100, centers[i][1] + 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                # cv.putText(frame, str(ang), (centers[i][0], centers[i][1] + 150), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                # cv.line(frame, (rim[0], rim[1]), (centers[i][0], centers[i][1]), (0, 255, 0), 2)
                if ang > np.pi/6 and ang < np.pi*5/6:
                    cv.putText(frame, "SHOT MADE", (centers[i][0], centers[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
                else:
                    cv.putText(frame, "SHOT MISSED", (centers[i][0], centers[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))



    # if len(rim) > 0:
    #     cv.circle(frame, (rim[0], rim[1]), int(rim[2] * 1.2), (0, 0, 255), 2)


    cv.imshow(name, frame)
    out.write(frame)
    return



# default values (good for orange)
val1 = 5
val2 = 15
val3 = 150
val4 = 255
val5 = 0
val6 = 255

# values good for rim
rimval1 = 0
rimval2 = 15
rimval3 = 0
rimval4 = 140
rimval5 = 40
rimval6 = 140

def change_val1(val):
    global val1
    val1 = val
def change_val2(val):
    global val2
    val2 = val
def change_val3(val):
    global val3
    val3 = val
def change_val4(val):
    global val4
    val4 = val
def change_val5(val):
    global val5
    val5 = val
def change_val6(val):
    global val6
    val6 = val

    
print(cv.getWindowImageRect(window))

cv.createTrackbar("H_lower", window, val1, 180, change_val1)
cv.createTrackbar("H_upper", window, val2, 180, change_val2)
cv.createTrackbar("S_lower", window, val3, 255, change_val3)
cv.createTrackbar("S_upper", window, val4, 255, change_val4)
cv.createTrackbar("V_lower", window, val5, 255, change_val5)
cv.createTrackbar("V_upper", window, val6, 255, change_val6)


dims = cv.getWindowImageRect(window)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter('output.mp4', fourcc, 20.0, (1280, 720))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
    # # mask = cv.inRange(hsv, orange_lower, orange_upper)
    # lower, upper = np.array([val1, val3, val5]), np.array([val2, val4, val6])
    # mask = cv.inRange(hsv, lower, upper)

    # result = cv.bitwise_and(frame, frame, mask = mask) 
    



    frame_counter += 1

    if frame_counter == 10:
        cv.imwrite("test.jpg", frame)

    try:
        perf_op(window, frame, centers, counter, rim_blobs)
    except Exception as e:
        print(e)
        break

    if cv.waitKey(1) == ord('q'):
        break
    if cv.waitKey(1) == ord('p'):
        cv.waitKey(-1)
 
# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()

(8, 31, 1280, 845)


C:\Users\Owner\AppData\Local\Temp\ipykernel_48272\205792057.py:114: RuntimeWarning: overflow encountered in scalar subtract
  elif ((centers[-1][0] - i[0])**2 + (centers[-1][1] - i[1])**2 )**.5 > centers[-1][2] * 4:
C:\Users\Owner\AppData\Local\Temp\ipykernel_48272\205792057.py:114: RuntimeWarning: invalid value encountered in power
  elif ((centers[-1][0] - i[0])**2 + (centers[-1][1] - i[1])**2 )**.5 > centers[-1][2] * 4:


Can't receive frame (stream end?). Exiting ...
